# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. 

### Query 1:   
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For this table, with this data we can use session_id and item_in_session as a composite primary key, as they are unique for every entry in the data. We can then query based in the session, and item in session. On top of those colums we'll use the ones we need for the query, artist, song title and the song's length 

#### Creating table 

In [8]:
query = "CREATE TABLE IF NOT EXISTS artist_title_length_per_session_and_item "
query = query+"(session_id int, item_in_session int, artist text, song_title text, song_length decimal, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

#### Populating the table with data
On top of this we have to take in account the format. Reading from a .csv file will take the data as strings, and we will use int() and float() to format this data into our table correctly. 


In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_title_length_per_session_and_item (session_id, item_in_session, artist, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to run the query

In [10]:
query = "SELECT artist, song_title, song_length FROM artist_title_length_per_session_and_item WHERE session_id=338 and item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Query 2: 
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To sort this query by iteminsession, we would have to add iteminsession to our key, so it's arranged by this column. We'll add userid to our key 

#### Creating table


In [18]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user_per_user_and_session "
query = query+"(user_id int, session_id int, item_in_session int, artist text, song_title text, user_name_full text, PRIMARY KEY ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

#### Populating the table with data

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO artist_song_user_per_user_and_session (user_id, session_id, item_in_session, artist, song_title, user_name_full) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        full_name = line[1]+" "+line[4]
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], full_name))

#### SELECT statement to run the query

In [20]:
query = "SELECT artist, song_title, user_name_full FROM artist_song_user_per_user_and_session WHERE user_id=10 and session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_title, row.user_name_full)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this case we will do the song title our primary key, combined with the user_

#### Creating table

In [25]:
query = "CREATE TABLE IF NOT EXISTS usernames_per_song "
query = query+"(song_title text, user_name_full text, PRIMARY KEY (song_title, user_name_full))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Populating the table with data

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO usernames_per_song (song_title, user_name_full) "
        query = query + "VALUES (%s, %s)"
        full_name = line[1]+" "+line[4]
        session.execute(query, (line[9], full_name))

#### SELECT statement to run the query

In [27]:
query = "SELECT user_name_full FROM usernames_per_song WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.user_name_full)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Dropping the tables

In [24]:
query = "DROP TABLE artist_title_length_per_session_and_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_song_user_per_user_and_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE usernames_per_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table artist_title_length_per_session_and_item"


### Closing the session

In [ ]:
session.shutdown()
cluster.shutdown()